In [3]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\user\anaconda3\python.exe -m pip install --upgrade pip' command.


In [4]:
pip install PyMuPDF

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\user\anaconda3\python.exe -m pip install --upgrade pip' command.


In [6]:
pip install gensim

     ---------------------------------------- 24.0/24.0 MB 9.4 MB/s eta 0:00:00
     ---------------------------------------- 56.8/56.8 KB 2.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\user\anaconda3\python.exe -m pip install --upgrade pip' command.


In [13]:
!pip install --upgrade numpy

You should consider upgrading via the 'C:\Users\user\anaconda3\python.exe -m pip install --upgrade pip' command.


In [14]:
!pip install --upgrade gensim

     --------------------------------------- 14.7/14.7 MB 10.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.1
    Uninstalling numpy-1.25.1:
      Successfully uninstalled numpy-1.25.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.3.0 requires daal==2021.2.3, which is not installed.
numba 0.54.1 requires numpy<1.21,>=1.17, but you have numpy 1.22.4 which is incompatible.
You should consider upgrading via the 'C:\Users\user\anaconda3\python.exe -m pip install --upgrade pip' command.


# 분산표현 기반 시소러스 구축(중첩리스트 각 문서 별)

In [7]:
import os
import gensim
import re
from nltk.corpus import stopwords

import numpy as np

stop_words = set(stopwords.words('english'))

synonyms = {}

corpus_folder_path = 'corpus_number_masked'

corpus = []

for file_name in os.listdir(corpus_folder_path):
    if file_name.endswith('.txt'):
        file_path = os.path.join(corpus_folder_path, file_name)
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
            words = re.findall(r'\w+', text.lower())  
            words = [word for word in words if word not in stop_words] 
            corpus.append(words)

model = gensim.models.Word2Vec(corpus, vector_size=300, window=5, min_count=20, workers=4, sg=1)

for word in model.wv.index_to_key:
    similar_words = model.wv.most_similar(word, topn=10)
    synonyms[word] = [(similar_word[0], similar_word[1]) for similar_word in similar_words]

output_file = '분산표현기반시소러스_사이즈300빈도20이상.txt'
with open(output_file, 'w', encoding='utf-8') as file:
    for word in synonyms:
        similar_words = synonyms[word]
        line = f'{word}: ' + ', '.join([f'{similar_word[0]}({similar_word[1]:.2f})' for similar_word in similar_words])
        file.write(line + '\n')
        

print("분산표현기반 시소러스 구축이 완료되었습니다.")


분산표현기반 시소러스 구축이 완료되었습니다.


# 분산표현 기반 시소러스(중첩 리스트 각 문장단위)

In [1]:
import os
import gensim
import re
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

synonyms = {}

corpus_folder_path = 'corpus_number_removed'

corpus_sentences = []

for file_name in os.listdir(corpus_folder_path):
    if file_name.endswith('.txt'):
        file_path = os.path.join(corpus_folder_path, file_name)
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
            sentences = re.split(r'[.!?]', text)  # 문단을 문장으로 분리
            for sentence in sentences:
                words = re.findall(r'\w+', sentence.lower())  # 문장을 단어로 분리 (구두점 제거)
                words = [word for word in words if word not in stop_words]  # 불용어 제거
                corpus_sentences.append(words)

model = gensim.models.Word2Vec(corpus_sentences, vector_size=200, window=5, min_count=15, workers=4, sg=1)

for word in model.wv.index_to_key:
    similar_words = model.wv.most_similar(word, topn=10)
    synonyms[word] = [(similar_word[0], similar_word[1]) for similar_word in similar_words]

output_file = '분산표현기반시소러스_문장단위20015.txt'
with open(output_file, 'w', encoding='utf-8') as file:
    for word in synonyms:
        similar_words = synonyms[word]
        line = f'{word}: ' + ', '.join([f'{similar_word[0]}({similar_word[1]:.2f})' for similar_word in similar_words])
        file.write(line + '\n')

print("분산표현기반 시소러스 구축이 완료되었습니다.")


분산표현기반 시소러스 구축이 완료되었습니다.


# 분산표현 기반 시소러스(중첩 리스트 각 문장단위 / 단어 깨지는 문제 개선)

In [18]:
import os
import gensim
import nltk
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

synonyms = {}

corpus_folder_path = 'corpus_number_removed'

corpus_sentences = []

for file_name in os.listdir(corpus_folder_path):
    if file_name.endswith('.txt'):
        file_path = os.path.join(corpus_folder_path, file_name)
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
            sentences = nltk.sent_tokenize(text) 
            for sentence in sentences:
                words = nltk.word_tokenize(sentence.lower())
                words = [word for word in words if word not in stop_words] 
                corpus_sentences.append(words)

model = gensim.models.Word2Vec(corpus_sentences, vector_size=200, window=5, min_count=15, workers=4, sg=1)

for word in model.wv.index_to_key:
    similar_words = model.wv.most_similar(word, topn=10)
    synonyms[word] = [(similar_word[0], similar_word[1]) for similar_word in similar_words]

output_file = '분산표현기반시소러스_문장단위20015_단어안깨지게.txt'
with open(output_file, 'w', encoding='utf-8') as file:
    for word in synonyms:
        similar_words = synonyms[word]
        line = f'{word}: ' + ', '.join([f'{similar_word[0]}({similar_word[1]:.2f})' for similar_word in similar_words])
        file.write(line + '\n')

print("분산표현기반 시소러스 구축이 완료되었습니다.")


분산표현기반 시소러스 구축이 완료되었습니다.


In [8]:
pip install networkx

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\user\anaconda3\python.exe -m pip install --upgrade pip' command.


In [9]:
pip install matplotlib

You should consider upgrading via the 'C:\Users\user\anaconda3\python.exe -m pip install --upgrade pip' command.


In [6]:
pip install pyvis

     -------------------------------------- 756.0/756.0 kB 4.3 MB/s eta 0:00:00
     ---------------------------------------- 40.5/40.5 kB 1.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


# 시소러스 유사도 n 이상

In [22]:
embedding_thesaurus_file = '분산표현기반시소러스_문장단위20015_단어안깨지게.txt'

new_embedding_thesaurus_file = '분산표현기반시소러스_0.9이상.txt'
selected_entries = [] 

with open(embedding_thesaurus_file, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()
        word, synonyms_str = line.split(': ', 1)
        synonyms_list = synonyms_str.split(', ')
        selected_synonyms = []
        for synonym in synonyms_list:
            synonym_word, similarity = synonym.split('(')
            similarity = float(similarity[:-1])
            if similarity >= 0.9:
                selected_synonyms.append(f"{synonym_word}({similarity})")
        if selected_synonyms:
            selected_entries.append(f"{word}: {', '.join(selected_synonyms)}")

with open(new_embedding_thesaurus_file, 'w', encoding='utf-8') as file:
    for entry in selected_entries:
        file.write(entry + '\n')

print("새로운 분산표현 기반 시소러스 파일이 생성되었습니다:", new_embedding_thesaurus_file)


새로운 분산표현 기반 시소러스 파일이 생성되었습니다: 분산표현기반시소러스_0.9이상.txt


# 시소러스 WordNetwork 시각화

In [23]:
import networkx as nx
from pyvis.network import Network

new_thesaurus_file = '분산표현기반시소러스_0.9이상.txt'

G = nx.Graph()

with open(new_thesaurus_file, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()
        word, synonyms_str = line.split(': ', 1)
        synonyms_list = synonyms_str.split(', ')
        G.add_node(word)  # 기준단어 노드 추가
        for synonym in synonyms_list:
            synonym_word, similarity = synonym.split('(')
            similarity = float(similarity[:-1])
            G.add_edge(word, synonym_word, weight=similarity)  

layout = nx.spring_layout(G, seed=42)

nt = Network(notebook=True, height='800px', width='100%')

for node in G.nodes():
    nt.add_node(node, label=node, title=node)

for edge in G.edges():
    source, target, weight = edge[0], edge[1], G.edges[edge]['weight']
    nt.add_edge(source, target, value=weight, title=f'Weight: {weight}')

output_file = "word2vec_thesaurus_network_0.9이상.html"


In [24]:
nt.show(output_file)

word2vec_thesaurus_network_0.9이상.html
